In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, StratifiedKFold

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Input, MaxPooling2D, Concatenate
from tensorflow.keras.layers import Conv1D, MaxPool1D, concatenate
from tensorflow.keras.optimizers import Adam
import joblib
import copy
import timeit

In [3]:
right_circle_df = pd.read_csv("./data/left_circle_6_points.csv")
left_circle_df = pd.read_csv("./data/right_circle_6_points.csv")
start_df = pd.read_csv("./data/start_6_points.csv")
random_movements_df = pd.read_csv("./data/random_movements_6_points.csv")

In [4]:
left_circle_df = left_circle_df.drop(columns=["Unnamed: 0"], axis=1)
right_circle_df = right_circle_df.drop(columns=["Unnamed: 0"], axis=1)
start_df = start_df.drop(columns=["Unnamed: 0"], axis=1)
random_movements_df = random_movements_df.drop(columns=["Unnamed: 0"], axis=1)

In [5]:
def to_array(df):
    x = []
    new_x = []
    for i in range(len(df.columns)):    
        x.append(df[str(i + 1)].to_numpy())

    for x_arr in x:
        x_arr_parsed = []
        for x_val in x_arr:
            x_arr_parsed.append(extract_array(x_val))
        new_x.append(x_arr_parsed)
        
    new_x = np.array(new_x)
    return np.array(new_x)

In [6]:
def extract_array(string_array):
    arr = string_array.replace('\'', '').replace('[', '').replace(']', '').replace(' ', '').split(',')
    arr = map(lambda a : float(a), arr) 
    return list(arr)

In [7]:
random_movements_x = to_array(random_movements_df)
random_movements_y = np.zeros(len(random_movements_x))

start_x = to_array(start_df)
start_y = np.ones(len(start_x))

left_circle_x = to_array(left_circle_df)
left_circle_y = np.full((left_circle_x.shape[0]), 2)

right_circle_x = to_array(right_circle_df)
right_circle_y = np.full((right_circle_x.shape[0]), 3)

print(left_circle_x.shape, right_circle_x.shape, start_x.shape, random_movements_x.shape)
print(left_circle_y.shape, right_circle_y.shape, start_y.shape, random_movements_y.shape)

x = np.concatenate((left_circle_x, right_circle_x, start_x, random_movements_x), axis = 0)
y = np.concatenate((left_circle_y, right_circle_y, start_y, random_movements_y), axis = 0)

(150, 150, 6) (188, 150, 6) (200, 150, 6) (201, 150, 6)
(150,) (188,) (200,) (201,)


In [8]:
def is_spike_up(threshold_up, threshold_down, x_window):
    spike = [None, None]

    for i in range(len(x_window) - 1):
        if x_window[i + 1] > x_window[i] * threshold_up:
            spike[0] = i
            
    if spike[0] != None:
        for i in range(spike[0], len(x_window) - 1):
            if x_window[i + 1] < x_window[i] * threshold_down:
                spike[1] = i
            
    return spike
            
def is_spike_down(threshold_up, threshold_down, x_window):
    spike = [None, None]

    for i in range(len(x_window) - 1):
        if x_window[i + 1] < x_window[i] * threshold_down:
            spike[0] = i
            
    if spike[0] != None:
        for i in range(spike[0], len(x_window) - 1):
            if x_window[i + 1] > x_window[i] * threshold_up:
                spike[1] = i
            
    return spike
        
def get_is_spike(x, x_window):
    threshold_up = 50
    threshold_down = 0.02
    is_spike = False
    spike = [None, None]
    
    spike_up = is_spike_up(threshold_up, threshold_down, x_window)
    spike_down = is_spike_down(threshold_up, threshold_down, x_window)
    
    if spike_up[0] != None and spike_up[1] != None:
        is_spike = True
        spike = spike_up
    elif spike_down[0] != None and spike_down[1] != None:
        is_spike = True
        spike = spike_down
        
    return (is_spike, spike)


def get_filterd_data(x):
    x = np.array(x)
    for i in range(len(x)):
        if i > 1 and i < len(x) - 2:
            for j in range(len(x[i])):
                x_window = x[i - 2 : i + 3][:, j] # need to be odd
                is_spike, spike_window_indexes = get_is_spike(x[i][j], x_window)
                if is_spike:
                    x[i][j] = np.median(x_window)
    return x

In [9]:
def normalize_data(x):
    scaler = MinMaxScaler()
    x_scaled = scaler.fit_transform(x)
    return x_scaled

In [10]:
for i in range(x.shape[0]):
    x[i] = get_filterd_data(x[i])
print(x.shape, y.shape)


(739, 150, 6) (739,)


In [11]:
folds = 5
seed = 666
models = []

kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

In [12]:
def get_model():
    flex_input = Input(shape=(150, 1))
    aDegX_input = Input(shape=(150, 1))
    aDegY_input = Input(shape=(150, 1))
    accX_input = Input(shape=(150, 1))
    accY_input = Input(shape=(150, 1))
    accZ_input = Input(shape=(150, 1))
    
    CNN_Layer_1 = 64
    
    CNN_Layers = [64, 64, 64, 64, 64, 64, 64]
    
    flex_CNN = Conv1D(filters = CNN_Layer_1, kernel_size = 3, activation = 'relu')(flex_input)
    aDegX_CNN = Conv1D(filters = CNN_Layer_1, kernel_size = 3, activation = 'relu')(aDegX_input)
    aDegY_CNN = Conv1D(filters = CNN_Layer_1, kernel_size = 3, activation = 'relu')(aDegY_input)
    accX_CNN = Conv1D(filters = CNN_Layer_1, kernel_size = 3, activation = 'relu')(accX_input)
    accY_CNN = Conv1D(filters = CNN_Layer_1, kernel_size = 3, activation = 'relu')(accY_input)
    accZ_CNN = Conv1D(filters = CNN_Layer_1, kernel_size = 3, activation = 'relu')(accZ_input)

    for i in range(len(CNN_Layers) - 1):
        flex_CNN = Conv1D(filters = CNN_Layers[i + 1], kernel_size = 3, activation = 'relu')(flex_CNN)
        aDegX_CNN = Conv1D(filters = CNN_Layers[i + 1], kernel_size = 3, activation = 'relu')(aDegX_CNN)
        aDegY_CNN = Conv1D(filters = CNN_Layers[i + 1], kernel_size = 3, activation = 'relu')(aDegY_CNN)
        accX_CNN = Conv1D(filters = CNN_Layers[i + 1], kernel_size = 3, activation = 'relu')(accX_CNN)
        accY_CNN = Conv1D(filters = CNN_Layers[i + 1], kernel_size = 3, activation = 'relu')(accY_CNN)
        accZ_CNN = Conv1D(filters = CNN_Layers[i + 1], kernel_size = 3, activation = 'relu')(accZ_CNN)

    flex_flatten = Flatten()(flex_CNN)
    aDegX_flatten = Flatten()(aDegX_CNN)
    aDegY_flatten = Flatten()(aDegY_CNN)
    accX_flatten = Flatten()(accX_CNN)
    accY_flatten = Flatten()(accY_CNN)
    accZ_flatten = Flatten()(accZ_CNN)
    
    flex_model = Model(inputs = flex_input, outputs = flex_flatten) 
    aDegX_model = Model(inputs = aDegX_input, outputs = aDegX_flatten) 
    aDegY_model = Model(inputs = aDegY_input, outputs = aDegY_flatten) 

    accX_model = Model(inputs = accX_input, outputs = accX_flatten) 
    accY_model = Model(inputs = accY_input, outputs = accY_flatten) 
    accZ_model = Model(inputs = accZ_input, outputs = accZ_flatten) 
    
    combined = concatenate([flex_model.output, aDegX_model.output, aDegY_model.output, accX_model.output, accY_model.output, accZ_model.output])
    dense_layer = Dense(256, activation = 'relu')(combined)
    dense_layer = Dense(64, activation = 'relu')(dense_layer)
    dense_layer = Dense(32, activation = 'relu')(dense_layer)
    
    output = Dense(4, activation='sigmoid')(dense_layer)
    
    model = Model(inputs = [flex_model.input, aDegX_model.input, aDegY_model.input, accX_model.input, accY_model.input, accZ_model.input], outputs = output)
    model.compile(optimizer = Adam(learning_rate = 0.001),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
    
    return model

In [13]:
def extract_array_by_sensors(x):
    to_return = []
    for i in range(x.shape[2]):
        acc_x = x[:, :, i]
   
        to_return.append(acc_x)

    return np.array(to_return)  

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=40)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 6)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 6)

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train.reshape(-1, x_train.shape[-1])).reshape(x_train.shape)
x_test = scaler.transform(x_test.reshape(-1, x_test.shape[-1])).reshape(x_test.shape)

x_train = extract_array_by_sensors(x_train)
x_test = extract_array_by_sensors(x_test)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)

print(x_train.shape)
model = get_model()
model.fit(x = [x_train[0], x_train[1], x_train[2], x_train[3], x_train[4], x_train[5]], y = y_train, epochs = 20, validation_data=([x_test[0], x_test[1], x_test[2], x_test[3], x_test[4], x_test[5]], y_test), verbose=1)

(6, 591, 150, 1)
Train on 591 samples, validate on 148 samples
Epoch 1/20
591/591 [==============================] - 7s 12ms/sample - loss: 1.3835 - accuracy: 0.2504 - val_loss: 1.3633 - val_accuracy: 0.4189
Epoch 2/20
591/591 [==============================] - 4s 7ms/sample - loss: 1.2872 - accuracy: 0.3892 - val_loss: 1.0394 - val_accuracy: 0.5676
Epoch 3/20
591/591 [==============================] - 5s 8ms/sample - loss: 0.9430 - accuracy: 0.5753 - val_loss: 0.8044 - val_accuracy: 0.6014
Epoch 4/20
591/591 [==============================] - 6s 10ms/sample - loss: 0.7825 - accuracy: 0.6904 - val_loss: 0.7555 - val_accuracy: 0.6824
Epoch 5/20
591/591 [==============================] - 5s 9ms/sample - loss: 0.6411 - accuracy: 0.7411 - val_loss: 0.7008 - val_accuracy: 0.7230
Epoch 6/20
591/591 [==============================] - 6s 10ms/sample - loss: 0.5725 - accuracy: 0.7766 - val_loss: 0.6933 - val_accuracy: 0.7297
Epoch 7/20
591/591 [==============================] - 5s 8ms/sample - 

In [15]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 1)]     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 150, 1)]     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 150, 1)]     0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 150, 1)]     0                                            
____________________________________________________________________________________________

In [17]:
model.save("./models/model_6_points.h5")
joblib.dump(scaler, './scalers/scaler.gz')

['./scalers/scaler.gz']